In [120]:
%matplotlib inline
import random
import time 
import torch
import torch.nn as nn
from torch.nn import init
import numpy as np
import pandas as pd
from IPython import display
from collections import OrderedDict
import sys
import d2lzh as d2l
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d


# 2 预备知识

## 2.2 数据操作

### 2.2.1创建 `tensor`

In [163]:
# 创建⼀个5x3的未初始化的 Tensor
x = torch.empty(5,3)
x

tensor([[1.0286e-38, 1.0653e-38, 1.0194e-38],
        [8.4490e-39, 1.0469e-38, 9.3674e-39],
        [9.9184e-39, 8.7245e-39, 9.2755e-39],
        [8.9082e-39, 9.9184e-39, 8.4490e-39],
        [9.6429e-39, 1.0653e-38, 1.0469e-38]])

In [164]:
# 创建⼀个5x3的随机初始化的 Tensor :
x = torch.rand(5,3)
x

tensor([[0.2475, 0.5785, 0.4930],
        [0.8624, 0.1975, 0.8793],
        [0.0370, 0.3336, 0.4633],
        [0.1396, 0.3209, 0.1564],
        [0.2538, 0.8468, 0.1671]])

In [165]:
# 创建⼀个5x3的long型全0的 Tensor :
x = torch.zeros(5,3,dtype = torch.long)
x

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [166]:
# 直接创建
x = torch.tensor([5.5,2])
x

tensor([5.5000, 2.0000])

In [167]:
# 通过现有的tensor插件，会默认重用输入tensor的一些属性，如数据类型等，除非自定义数据类型
x = x.new_ones(5, 3, dtype = torch.float64)
# 返回的tensor默认具有相同的torch.dtype和torch.device
x

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)

In [168]:
# 制定新的数据类型
x = torch.randn_like(x, dtype=torch.float)
x

tensor([[-1.2324, -0.3250,  0.4171],
        [ 0.4368,  1.0193,  0.8029],
        [-0.1245,  0.3772, -0.1379],
        [-1.1019,  2.3555, -0.2890],
        [-1.9084, -0.9716, -0.7206]])

In [169]:
# 可以通过哦shape,size()来获取tensor的形状
x.size(),x.shape

(torch.Size([5, 3]), torch.Size([5, 3]))

> 注意：返回的torch.Size其实就是一个tuple, 支持所有tuple的操作。 

还有很多函数可以创建`Tensor`，去翻翻官方API就知道了，下表给了一些常用的作参考。

|函数|功能|
|:---:|:---:|
|Tensor(*sizes)|基础构造函数|
|tensor(data,)|类似np.array的构造函数|
|ones(*sizes)|全1Tensor|
|zeros(*sizes)|全0Tensor|
|eye(*sizes)|对角线为1，其他为0|
|arange(s,e,step)|从s到e，步长为step|
|linspace(s,e,steps)|从s到e，均匀切分成steps份|
|rand/randn(*sizes)|均匀/标准分布|
|normal(mean,std)/uniform(from,to)|正态分布/均匀分布|
|randperm(m)|随机排列|

这些创建方法都可以在创建的时候指定数据类型dtype和存放device(cpu/gpu)。

### 2.2.2 操作

#### 加法

In [170]:
y = torch.rand(5,3)
y = torch.rand(5,3)

# 加法形式1：
print(x + y )

# 加法形式2：
print(torch.add(x,y))
#还可以指定输出
res = torch.empty(5,3)
torch.add(x,y,out=res)
print(res)

# 加法形式3：
y.add_(x)
y

tensor([[-0.4729,  0.6191,  1.3751],
        [ 0.7469,  1.6783,  1.0127],
        [ 0.4769,  1.0914, -0.1163],
        [-0.6928,  2.6008,  0.2075],
        [-1.3178, -0.4104, -0.1003]])
tensor([[-0.4729,  0.6191,  1.3751],
        [ 0.7469,  1.6783,  1.0127],
        [ 0.4769,  1.0914, -0.1163],
        [-0.6928,  2.6008,  0.2075],
        [-1.3178, -0.4104, -0.1003]])
tensor([[-0.4729,  0.6191,  1.3751],
        [ 0.7469,  1.6783,  1.0127],
        [ 0.4769,  1.0914, -0.1163],
        [-0.6928,  2.6008,  0.2075],
        [-1.3178, -0.4104, -0.1003]])


tensor([[-0.4729,  0.6191,  1.3751],
        [ 0.7469,  1.6783,  1.0127],
        [ 0.4769,  1.0914, -0.1163],
        [-0.6928,  2.6008,  0.2075],
        [-1.3178, -0.4104, -0.1003]])

#### 索引

In [171]:
# 索引出来的结果与原数据共享内存，也即修改⼀个，另⼀个会跟着修改
y = x[0,:]
y += 1

y,x[0,:]

(tensor([-0.2324,  0.6750,  1.4171]), tensor([-0.2324,  0.6750,  1.4171]))

In [172]:
y = torch.index_select(x,1,2,*,y)
y

SyntaxError: invalid syntax (<ipython-input-172-a7743ddc4e52>, line 1)

除了常用的索引选择数据之外，PyTorch还提供了一些高级的选择函数:

|函数|	功能|
|:---:|:---:|
|index_select(input, dim, index)|在指定维度dim上选取，比如选取某些行、某些列|
|masked_select(input, mask)|例子如上，a[a>0]，使用ByteTensor进行选取|
|nonzero(input)|	非0元素的下标|
|gather(input, dim, index)|根据index，在dim维度上选取数据，输出的size与index一样|

这里不详细介绍，用到了再查官方文档。

#### 改变形状

In [173]:
# view()返回的新tensor与源tensor共享内存
# view仅改变了观察方式
y = x.view(15)
z = x.view(-1,5)

x.size(),y.size(),z.size()

(torch.Size([5, 3]), torch.Size([15]), torch.Size([3, 5]))

In [174]:
x += 1

print(x)
print(y)

tensor([[ 0.7676,  1.6750,  2.4171],
        [ 1.4368,  2.0193,  1.8029],
        [ 0.8755,  1.3772,  0.8621],
        [-0.1019,  3.3555,  0.7110],
        [-0.9084,  0.0284,  0.2794]])
tensor([ 0.7676,  1.6750,  2.4171,  1.4368,  2.0193,  1.8029,  0.8755,  1.3772,
         0.8621, -0.1019,  3.3555,  0.7110, -0.9084,  0.0284,  0.2794])


In [175]:
# 如果我们想返回⼀个真正新的副本（即不共享内存）
#Pytorch还提供了⼀个 reshape() 可以改变形状，但是此函数并不能保证返回的是其拷⻉，所以不推荐使⽤
# 推荐先⽤ clone 创造⼀个副本然后再使⽤ view

x_cp = x.clone().view(15)
x -= 1

print(x)
print(x_cp)

tensor([[-0.2324,  0.6750,  1.4171],
        [ 0.4368,  1.0193,  0.8029],
        [-0.1245,  0.3772, -0.1379],
        [-1.1019,  2.3555, -0.2890],
        [-1.9084, -0.9716, -0.7206]])
tensor([ 0.7676,  1.6750,  2.4171,  1.4368,  2.0193,  1.8029,  0.8755,  1.3772,
         0.8621, -0.1019,  3.3555,  0.7110, -0.9084,  0.0284,  0.2794])


> 使用`clone`还有一个好处是会被记录在计算图中，即梯度回传到副本时也会传到源`Tensor`。

In [176]:
# item()将一个标量tensor转变成numpy
# only one element tensors can be converted to Python scalars
x = torch.randn(1)
x,x.item()

(tensor([0.8057]), 0.8057039380073547)

#### 线性代数
另外，PyTorch还支持一些线性函数，这里提一下，免得用起来的时候自己造轮子，具体用法参考官方文档。如下表所示：

| 函数	|功能|
|:---:|:---:|
|trace|	对角线元素之和(矩阵的迹)|
|diag|	对角线元素|
|triu/tril	|矩阵的上三角/下三角，可指定偏移量|
|mm/bmm	|矩阵乘法，batch的矩阵乘法|
|addmm/addbmm/addmv/addr/baddbmm..|	矩阵运算|
|t|转置|
|dot/cross|	内积/外积|
|inverse	|求逆矩阵|
|svd	|奇异值分解|

### 2.2.3 广播机制

前面我们看到如何对两个形状相同的`Tensor`做按元素运算。

当对两个形状不同的`Tensor`按元素运算时，可能会触发广播（broadcasting）机制：

先适当复制元素使这两个`Tensor`形状相同后再按元素运算。

In [177]:
x = torch.arange(1, 3).view(1, 2)
print(x)
y = torch.arange(1, 4).view(3, 1)
print(y)
print(x + y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


由于 x 和 y 分别是1⾏2列和3⾏1列的矩阵，如果要计算 x + y ，那么 x 中第⼀⾏的2个元素被⼴播
（复制）到了第⼆⾏和第三⾏，⽽ y 中第⼀列的3个元素被⼴播（复制）到了第⼆列。如此，就可以对2
个3⾏2列的矩阵按元素相加

### 2.2.4运算的内存开销

前⾯说了，索引、 view 是不会开辟新内存的，⽽像 y = x + y 这样的运算是会新开内存的，然后将 y 指向新内存为了演示这⼀点，我们可以使⽤Python⾃带的 id 函数：如果两个实例的ID⼀致，那么它们所对应的内存地址相同；反之则不同

In [178]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
y_id = id(y)
y = y + x
y_id_ = id(y)

y_id,y_id_,y_id - y_id_

(1690813872512, 1690813871872, 640)

如果想指定结果到原来的 y 的内存，我们可以使⽤前⾯介绍的索引来进⾏替换操作。在下⾯的例⼦中，我们把 x + y 的结果通过 [:] 写进 y 对应的内存中。

In [179]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
y_id = id(y)
y[:] = y + x
y_id_ = id(y)

y_id,y_id_,y_id - y_id_

(1690815021568, 1690815021568, 0)

我们还可以使⽤运算符全名函数中的 out 参数或者⾃加运算符 += (也即 add_() )达到上述效果，例如
torch.add(x, y, out=y) 和 y += x ( y.add_(x) )。

In [180]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
y_id = id(y)
torch.add(x,y,out=y)# or y += x, y.add_(x)
y_id_ = id(y)

y_id,y_id_,y_id == y_id

(1690815021952, 1690815021952, True)

### 2.2.5 TENSOR 和NUMPY相互转换

Notes: tensor和numpy中的数组也共享内存,（所以他们之间的转换很
快），改变其中⼀个时另⼀个也会改变！！

#### numpy中的array转换成tensor
需要注意的是，此⽅法总是会进⾏数据拷⻉（就会消耗更多的时间和空间），所以返回的 Tensor 和原来的数
据不再共享内存。


In [181]:
a = np.ones(3)
b = torch.from_numpy(a)

a += 1
a,b

(array([2., 2., 2.]), tensor([2., 2., 2.], dtype=torch.float64))

In [182]:
# 法2
b = torch.tensor(a)

a,b

(array([2., 2., 2.]), tensor([2., 2., 2.], dtype=torch.float64))

所有在CPU上的 Tensor （除了 CharTensor ）都⽀持与NumPy数组相互转换

#### tensor转numpy

In [183]:
a = torch.ones(2)
b = a.numpy()

a += 1

a,b

(tensor([2., 2.]), array([2., 2.], dtype=float32))

### 2.2.6 TENSOR ON GPU
用方法`to()`可以将`Tensor`在CPU和GPU（需要硬件支持）之间相互移动。


In [184]:
# 以下代码只有在PyTorch GPU版本上才会执行
if torch.cuda.is_available():
    device = torch.device("cuda")          # GPU
    y = torch.ones_like(x, device=device)  # 直接创建一个在GPU上的Tensor
    x = x.to(device)                       # 等价于 .to("cuda")
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # to()还可以同时更改数据类型

## 2.3 ⾃动求梯度

在深度学习中，我们经常需要对函数求梯度（gradient）。PyTorch提供的autograd 包能够根据输⼊
和前向传播过程⾃动构建计算图，并执⾏反向传播。

### 2.3.1 概念

上一节介绍的`Tensor`是这个包的核心类，如果将其属性`.requires_grad`设置为`True`，它将开始追踪(track)在其上的所有操作（这样就可以利用链式法则进行梯度传播了）。完成计算后，可以调用`.backward()`来完成所有梯度计算。此`Tensor`的梯度将累积到`.grad`属性中。
> 注意在`y.backward()`时，如果`y`是标量，则不需要为`backward()`传入任何参数；否则，需要传入一个与`y`同形的`Tensor`。解释见 2.3.2 节。

如果不想要被继续追踪，可以调⽤ .detach() 将其从追踪记录中分离出来，这样就可以防⽌将来的计算被追踪，这样梯度就传不过去了。此外，还可以⽤ with torch.no_grad() 将不想被追踪的操作代码块包裹起来，这种⽅法在评估模型的时候很常⽤，因为在评估模型时，我们并不需要计算可训练参数（ requires_grad=True ）的梯度。

Function 是另外⼀个很᯿要的类。 Tensor 和 Function 互相结合就可以构建⼀个记录有整个计算过程的有向⽆环图（DAG）。每个 Tensor 都有⼀个 .grad_fn 属性，该属性即创建该 Tensor 的Function , 就是说该 Tensor 是不是通过某些运算得到的，若是，则 grad_fn 返回⼀个与这些运算相关的对象，否则是None。

### 2.3.2 `Tensor`

In [185]:
# 即把x当做变量
x = torch.ones(2,2,requires_grad = True)
x,x.grad_fn

(tensor([[1., 1.],
         [1., 1.]], requires_grad=True),
 None)

In [186]:
# 做一下运算

y = x +2
y,y.grad_fn

(tensor([[3., 3.],
         [3., 3.]], grad_fn=<AddBackward0>),
 <AddBackward0 at 0x189ac97ca60>)

注意x是直接创建的，所以它没有 grad_fn , ⽽y是通过⼀个加法操作创建的，所以它有⼀个为
<AddBackward> 的 grad_fn 

In [187]:
# 像x这种直接创建的称为叶⼦节点，叶⼦节点对应的 grad_fn 是 None 。
print(x.is_leaf,y.is_leaf)

True False


In [188]:
# 可以从grad_fn看出本次的操作 or 运算
z = y ** 2 * 3
out = z.mean()

print(z)
print(z.grad_fn)
print()
print(out)
print(out.grad_fn)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)

tensor(27., grad_fn=<MeanBackward0>)


通过`.requires_grad_()`来用in-place的方式改变`requires_grad`属性：


In [189]:
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)

False
True


### 2.3.3 梯度


In [190]:
#  因为 out 是⼀个标量，所以调⽤ backward() 时不需要指定求导变量
out.backward()

 我们来看看`out`关于`x`的梯度 $\frac{d(out)}{dx}$:


In [191]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


我们令`out`为 $o$ , 因为
$$
o=\frac14\sum_{i=1}^4z_i=\frac14\sum_{i=1}^43(x_i+2)^2
$$
所以
$$
\frac{\partial{o}}{\partial{x_i}}\bigr\rvert_{x_i=1}=\frac{9}{2}=4.5
$$
所以上面的输出是正确的。

数学上，如果有一个函数值和自变量都为向量的函数 $\vec{y}=f(\vec{x})$, 那么 $\vec{y}$ 关于 $\vec{x}$ 的梯度就是一个雅可比矩阵（Jacobian matrix）:
$$
J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)
$$
而``torch.autograd``这个包就是用来计算一些雅克比矩阵的乘积的。例如，如果 $v$ 是一个标量函数的 $l=g\left(\vec{y}\right)$ 的梯度：
$$
v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)
$$
那么根据链式法则我们有 $l$ 关于 $\vec{x}$ 的雅克比矩阵就为:
$$
v J=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right) \left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)=\left(\begin{array}{ccc}\frac{\partial l}{\partial x_{1}} & \cdots & \frac{\partial l}{\partial x_{n}}\end{array}\right)
$$

注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。

In [192]:
#再来一次反向传播，注意grad是累加的

out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])



> 现在我们解释2.3.1节留下的问题，为什么在`y.backward()`时，如果`y`是标量，则不需要为`backward()`传入任何参数；否则，需要传入一个与`y`同形的`Tensor`?
简单来说就是为了避免向量（甚至更高维张量）对张量求导，而转换成标量对张量求导。举个例子，假设形状为 `m x n` 的矩阵 X 经过运算得到了 `p x q` 的矩阵 Y，Y 又经过运算得到了 `s x t` 的矩阵 Z。那么按照前面讲的规则，dZ/dY 应该是一个 `s x t x p x q` 四维张量，dY/dX 是一个 `p x q x m x n`的四维张量。问题来了，怎样反向传播？怎样将两个四维张量相乘？？？这要怎么乘？？？就算能解决两个四维张量怎么乘的问题，四维和三维的张量又怎么乘？导数的导数又怎么求，这一连串的问题，感觉要疯掉…… 
为了避免这个问题，我们**不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量**。所以必要时我们要**把张量通过将所有张量的元素加权求和的方式转换为标量**，举个例子，假设`y`由自变量`x`计算而来，`w`是和`y`同形的张量，则`y.backward(w)`的含义是：先计算`l = torch.sum(y * w)`，则`l`是个标量，然后求`l`对自变量`x`的导数。
[参考](https://zhuanlan.zhihu.com/p/29923090)

In [193]:
#Only Tensors of floating point and complex dtype can require gradients
x = torch.tensor([1.,2.,3.,4.],requires_grad=True)
y = 2 * x
z = y.view(2,2)
z

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)

现在 z 不是⼀个标量，所以在调⽤ backward 时需要传⼊⼀个和z同形的权重向量进⾏加权求和得到
⼀个标量

In [194]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)
# 注意， x.grad 是和 x 同形的张量

tensor([2.0000, 0.2000, 0.0200, 0.0020])


**为什么**

In [195]:
# for question
l = torch.sum(v * z)

In [196]:
# 再来卡看中断梯度的例子：
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

y3.backward()
print(x.grad) # 只算了y1 part

y2.backward() # 因为requires_grad = false，所以报错

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True
tensor(2.)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

此外，如果我们想要修改 tensor 的数值，但是⼜不希望被 autograd 记录（即不会影响反向传播），
那么我么可以对 tensor.data 进⾏操作。

In [ ]:
x = torch.ones(1,requires_grad=True)
print(x.data) # 还是⼀个tensor
print(x.data.requires_grad) # 但是已经是独⽴于计算图之外
y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播
y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
